## Data Annotation for Classification

TODO

In [ ]:
# General Imports.
import json
import os
import imageio.v3 as imageio

import magicgui
import napari
import numpy as np

from pathlib import Path
from skimage.measure import regionprops

In [ ]:
# This function will download and unpack the data and do some further data preparation.
# It will only be executed if the data has not been downloaded yet.
data_dir = "../data"
if os.path.exists(data_dir):
    print("The data is downloaded already.")
else:
    utils.prepare_data(data_dir)

In [ ]:
# Load data for an example image.

image_id = 0

image_path = os.path.join(data_dir, "train", f"gt_image_00{image_id}", f"gt_image_00{image_id}_serum_image.tif")
serum = imageio.imread(image_path)

image_path = os.path.join(data_dir, "train", f"gt_image_00{image_id}", f"gt_image_00{image_id}_marker_image.tif")
marker = imageio.imread(image_path)

seg_path = os.path.join(data_dir, "train", f"gt_image_00{image_id}", f"gt_image_00{image_id}_cell_labels.tif")
seg = imageio.imread(seg_path)

In [ ]:
# Helper Functions

def unique_labels(mask, label_data):
    return np.unique(label_data[mask])


def accumulate_labels(segmentation, label_data):
    props = regionprops(segmentation, label_data, extra_properties=[unique_labels])
    label_list = []
    
    for prop in props:
        mapped_labels = prop.unique_labels
        
        # Check how many labels we have mapped to this mask.
        # If we have more than one label mapped (excluding zero), then we treat this as non-labeled
        # because we cannot be sure which of the labels is correct.
        mapped_labels = np.setdiff1d(mapped_labels, [0])
        
        if len(mapped_labels) == 0:
            this_label = 0
        elif len(mapped_labels) == 1:
            this_label = mapped_labels[0]
        else:
            this_label = 0
        
        label_list.append({"cell_id": prop.label, "label": int(this_label), "bbox": prop.bbox})
        
    return label_list


def create_labels_for_display(segmentation, label_list):
    label_map = {lab["cell_id"]: lab["label"] for lab in label_list}
    out = np.zeros_like(segmentation)
    for cell_id, label in label_map.items():
        if label == 0:  # don't need to write out zero!
            continue
        out[segmentation == cell_id] = label
    return out

Explain `magicgui`.

https://napari.org/stable/guides/magicgui.html

In [ ]:
viewer = napari.Viewer()
viewer.add_image(serum, colormap="green", blending="additive")
viewer.add_image(marker, colormap="red", blending="additive")
seg_layer = viewer.add_labels(seg, name="segmentation")
seg_layer.contour = 2

viewer.add_labels(np.zeros_like(seg), name="labels")
viewer.add_labels(np.zeros_like(seg), name="mapped labels (visualization)")


@magicgui.magicgui(call_button="Display and export labels")
def export_labels(viewer: napari.Viewer, export_path: Path):

    layers = viewer.layers
    seg, label_data = layers["segmentation"].data, layers["labels"].data
    label_list = accumulate_labels(seg, label_data)

    for_vis = create_labels_for_display(seg, label_list)
    layers["mapped labels (visualization)"].data = for_vis
    layers["mapped labels (visualization)"].refresh()
    
    with open(export_path, "w") as f:
        json.dump({"cells": label_list}, f)


viewer.window.add_dock_widget(export_labels)

Explain the task.